# FINAL PROJECT - RESEARCH QUESTION 3

In [3]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from collections import Counter
from config import *
import implicit

## 1. LOAD DATA

In [4]:
%%time

with open("data/tweets.json", "rb") as f:
    data = f.readlines()
    data = [json.loads(str_) for str_ in data]

CPU times: user 1min 40s, sys: 7min 36s, total: 9min 17s
Wall time: 31min 50s


In [5]:
%%time
df_tweets = pd.DataFrame.from_records(data)

CPU times: user 19 s, sys: 2min 11s, total: 2min 30s
Wall time: 13min 31s


In [6]:
df_tweets_100 = df_tweets[:100000]

## 2. CREATE THE RETWEET GRAPH

In [7]:
from igraph import *

In [8]:
# Retweet graph 
df_retweets = df_tweets_100[~df_tweets_100['retweeted_status'].isnull()]

In [9]:
df_graph = pd.DataFrame(columns = ["source", "destination"])

# add source-nodes
df_graph["source"] = df_retweets['user'].apply(lambda x: x['screen_name'])

# add destination-nodes
df_graph["destination"] =  df_retweets['retweeted_status'].apply(lambda x: x['user']['screen_name'])

In [10]:
df_graph.drop_duplicates(inplace = True)

In [11]:
tuples = [tuple(x) for x in df_graph.values]
graph = Graph.TupleList(tuples, directed = False)

In [12]:
print(len(graph.es))

64203


## 3. TESTING ALGORITHMS USING THE FIRST OPTION

In [13]:
# fraction of edges to select as test-set
p = 0.2

# graphsize
N = len(graph.es)

# idxs of all the edges
all_idxs = range(N)

# sample idxs of edges through the function "choice"
test_idxs = np.random.choice(a = all_idxs, size = int(p*N), replace = False)

print(len(test_idxs))

12840


Remove the test edges from the graph

In [14]:
graph_del = Graph.TupleList(tuples, directed = False)
graph_del.delete_edges(test_idxs)
print(len(graph_del.es))

51363


In [15]:
ground_truth = set()
trainset = set()

for idx, one_edge in enumerate(graph.es):

    # take n1 and n2 idx from one_edge, that is an igraph edge *object*
    n1 = one_edge.source
    n2 = one_edge.target

    if idx in test_idxs:
        ground_truth.add((n1, n2, 1))
        
    else:
        trainset.add((n1, n2, 1))

In [16]:
unique_nodes = set()
for n1, n2, _ in ground_truth:
    unique_nodes.add(n1)
    unique_nodes.add(n2)

len(unique_nodes)

15455

In [17]:
def find_nodes_at_distance_2(graph, nodes):
    """
    starting from a graph this function returns all the nodes at distance 2
    """
    
    all_potential_recommendations = set()
    
    for n1 in nodes:
        
        # all the nodes at distance 1
        nodes_at_most_distant_1 = set(graph.neighborhood(vertices = n1, order = 1))

        # all the nodes at distance 1 and distance 2

        nodes_at_most_distant_2 = set(graph.neighborhood(vertices = n1, order = 2))
        
        # only the nodes at distance 2
        only_nodes_at_distance_2 = nodes_at_most_distant_2 - nodes_at_most_distant_1
        
        
        # check if empty set
        if len(only_nodes_at_distance_2) > 0:

            for n2 in only_nodes_at_distance_2:
                
                all_potential_recommendations.add((n1, n2))
            
    return all_potential_recommendations

In [18]:
%%time
all_potential_recommendations = find_nodes_at_distance_2(graph_del, unique_nodes)

CPU times: user 2.7 s, sys: 2.97 s, total: 5.67 s
Wall time: 26.2 s


In [19]:
len(all_potential_recommendations)

1068330

In [20]:
for rec in all_potential_recommendations:
    
    # add to ground truth also the potential nodes
    n1 = rec[0]
    n2 = rec[1]
    
    ground_truth.add((n1, n2, 0))

In [21]:
print(len(ground_truth))

1081170


### ALS

In [22]:
# first we get the adjacency matrix of tweets
M = graph_del.get_adjacency_sparse()

In [23]:
# here we run the model ALS
model = implicit.als.AlternatingLeastSquares(factors = 10, calculate_training_loss = True, iterations = 5)

# train the model on a sparse matrix of item/user/confidence weights
model.fit(M)

In [24]:
def predict_ALS(testset, model):
    """
    predict for a list of observations the score for adding/removing a link
    """

    # initialize the empty list
    all_predictions = []

    # scroll the obs
    for n1, n2, w in testset:
        
        # take here the low-dimensional vectors returned by the matrix factorization
        
        array_n1 = model.user_factors[n1,:]
        array_n2 = model.item_factors[n2,:]

        # multiplying these vectors we generate an approximation for the edge score
        one_p = np.dot(array_n1, array_n2)

        all_predictions.append(one_p)
        
    return all_predictions

In [25]:
# generate the predictions
df_test = pd.DataFrame(list(ground_truth), columns = ["n1", "n2", "edge"])
all_predictions = predict_ALS(df_test.values, model)

# add predictions to df
df_test["rating"] = all_predictions

# convert predictions to binary values: 0 don't add the edge, 1 add it.
df_test["rating"] = df_test["rating"].apply(lambda x: round(x))

In [26]:
# number of observations matched by the prediction
right_predictions = len(df_test[df_test['rating'] == df_test['edge']])

# accuracy
right_predictions/len(df_test)

0.9881054783244079

### ADAMIC ADAR

In [27]:
def compute_ADA(u, v, graph):
    """
    compute adamic-adar from scratch
    """
    
    # set of neighbors of u
    outlinks_from_u = graph.neighbors(u)

    # set of neighbors of v
    inlinks_to_v = graph.neighbors(v)

    # set Z of neighbors of both
    bridges = set(outlinks_from_u).intersection(inlinks_to_v)

    # degree of nodes in set Z
    deg_ = [graph.degree(n) for n in bridges]
    
    # computing the reciprocal in log-scale
    out = [1./np.log2(dd+1) for dd in deg_]

    return sum(out)

### JACCARD

In [28]:
def compute_Jaccard(u,v, graph):
    """
    compute jaccard similarity
    """
    

    # set of neighbors of u
    outlinks_from_u = graph.neighbors(u)

    # set of neighbors of v
    inlinks_to_v = graph.neighbors(v)

    # intesection of the two sets
    num = set(outlinks_from_u).intersection(inlinks_to_v)
    
    # union of the two sets
    den = set(outlinks_from_u).union(inlinks_to_v)
    
    # final division
    if len(den) == 0:
        out = 0
    else:
        out = len(num)/len(den)
    
    return out

### PAGERANK

In [29]:
# Get the unique nodes
unique_nodes = set()
for n1, n2, _ in ground_truth:
    unique_nodes.add(n1)
    unique_nodes.add(n2)

# Subset of 100 nodes
test_nodes = list(unique_nodes)[:100]

In [30]:
# Edges in the test set involving the test_nodes
test_ground_truth = set()

for n1, n2, val in ground_truth:
    if n1 in test_nodes or n2 in test_nodes:
        test_ground_truth.add((n1, n2, val))

In [31]:
print(len(test_nodes))
print(len(test_ground_truth))

100
10841


In [32]:
%%time
topk = 10

# Dataframe to store the top10 for each node
ADA_results = pd.DataFrame()
Jaccard_results = pd.DataFrame()
PageRank_results = pd.DataFrame()

# Iterate over the nodes in the test set
for node in test_nodes:
    
    # Create temporal datasets
    ADA_tmp = pd.DataFrame()
    Jaccard_tmp = pd.DataFrame()
    
    # Iterate over the test_ground_truth set
    for n1, n2, val in test_ground_truth:
        
        # Comptue the predicitons for the algorithms: ADA, Jaccard
        if node == n1:
            ADA_tmp = ADA_tmp.append({'source': node, 'destination': n2, 'true': val,
                                      'ADA_pred': compute_ADA(node, n2, graph_del)}, ignore_index = True)
            Jaccard_tmp = Jaccard_tmp.append({'source': node, 'destination': n2, 'true': val, 
                                              'Jaccard_pred': compute_Jaccard(node, n2, graph_del)}, 
                                             ignore_index = True)
            
        elif node == n2:
            ADA_tmp = ADA_tmp.append({'source': node, 'destination': n1, 'true': val, 
                                      'ADA_pred': compute_ADA(n1, node, graph_del)}, ignore_index = True)
            Jaccard_tmp = Jaccard_tmp.append({'source': node, 'destination': n1, 'true': val, 
                                              'Jaccard_pred': compute_Jaccard(n1, node, graph_del)}, 
                                             ignore_index = True)
    
    # Store the top10 
    ADA_tmp.sort_values(by = ['ADA_pred'], inplace = True, ascending = False)
    ADA_results = ADA_results.append(ADA_tmp[:topk])
    
    Jaccard_tmp.sort_values(by = ['Jaccard_pred'], inplace = True, ascending = False)
    Jaccard_results = Jaccard_results.append(Jaccard_tmp[:topk])
    
    # PageRank algorithm
    pr = enumerate(graph_del.personalized_pagerank(reset_vertices = node))
    out = sorted(pr, key = lambda x: x[1], reverse = True)
    
    i = 0
    for node2, pagerank in out:
        if i < topk and node != node2:
            for n1, n2, val in test_ground_truth:
                if node2 == n1 or node2 == n2:
                    i = i + 1
                    PageRank_results = PageRank_results.append({'source': node, 'destination': node2, 'true': val, 
                                                                'PageRank_pred': pagerank}, ignore_index = True)
                    break

CPU times: user 50.8 s, sys: 1.58 s, total: 52.3 s
Wall time: 59.1 s


In [33]:
print(ADA_results.head())

    ADA_pred  destination  source  true
0   0.141738       2084.0     2.0   0.0
1   0.141738      11719.0     2.0   0.0
30  0.141738       7048.0     2.0   0.0
29  0.141738       6370.0     2.0   0.0
28  0.141738      39355.0     2.0   0.0


In [34]:
print(Jaccard_results.head())

    Jaccard_pred  destination  source  true
19           1.0      55140.0     2.0   0.0
18           1.0      47586.0     2.0   0.0
24           1.0      35741.0     2.0   0.0
30           1.0       7048.0     2.0   0.0
6            1.0        976.0     2.0   0.0


In [35]:
print(PageRank_results.head())

   PageRank_pred  destination  source  true
0       0.239992          3.0     2.0   0.0
1       0.016956         31.0     2.0   1.0
2       0.014829        153.0     2.0   0.0
3       0.010863         27.0     2.0   0.0
4       0.006428        795.0     2.0   0.0


### NDCG SCORE

In [36]:
from sklearn.metrics import ndcg_score
from sklearn.metrics import average_precision_score

In [37]:
# NDCG score computation
ADA_score_nDCG = dict()
Jaccard_score_nDCG = dict()
PageRank_score_nDCG = dict()


for node in test_nodes:
    if len(ADA_results[ADA_results['source'] == node]) > 1:
        ADA_score_nDCG[node] = ndcg_score(np.asarray([ADA_results[ADA_results['source'] == node]['true'].values]), 
                                          np.asarray([ADA_results[ADA_results['source'] == node]['ADA_pred'].values]))
    
    if len(Jaccard_results[Jaccard_results['source'] == node]) > 1:
        Jaccard_score_nDCG[node] = ndcg_score(np.asarray([Jaccard_results[Jaccard_results['source'] == node]['true'].values]), 
                                              np.asarray([Jaccard_results[Jaccard_results['source'] == node]['Jaccard_pred'].values]))
    
    if len(PageRank_results[PageRank_results['source'] == node]) > 1:
        PageRank_score_nDCG[node] = ndcg_score(np.asarray([PageRank_results[PageRank_results['source'] == node]['true'].values]), 
                                               np.asarray([PageRank_results[PageRank_results['source'] == node]['PageRank_pred']]))

print('The scores for nDCG metrics are:\n')
print('ADA score %.4f' %(sum(ADA_score_nDCG.values())/len(ADA_score_nDCG.values())))
print('Jaccard score %.4f' %(sum(Jaccard_score_nDCG.values())/len(Jaccard_score_nDCG.values())))
print('PageRank score %.4f' %(sum(PageRank_score_nDCG.values())/len(PageRank_score_nDCG.values())))

The scores for nDCG metrics are:

ADA score 0.0815
Jaccard score 0.0865
PageRank score 0.3238


## 4. TESTING ALGORITHMS USING THE SECOND OPTION

In [38]:
# fraction of edges to select as test-set
p = 0.2

# graphsize
N = len(graph.es)

# idxs of all the edges
all_idxs = range(N)

# sample idxs of edges through the function "choice"
test_idxs = np.random.choice(a = all_idxs, size = int(p*N), replace = False)

In [39]:
# Remove from the graph the edges on the test set
graph_del = Graph.TupleList(tuples, directed = False)
graph_del.delete_edges(test_idxs)
print(len(graph_del.es))

51363


In [40]:
ground_truth = set()
trainset = set()

for idx, one_edge in enumerate(graph.es):

    # take n1 and n2 idx from one_edge, that is an igraph edge *object*
    n1 = one_edge.source
    n2 = one_edge.target

    if idx in test_idxs:
        ground_truth.add((n1, n2, 1))
        
    else:
        trainset.add((n1, n2, 1))

In [41]:
unique_nodes = set()
for n1, n2, _ in ground_truth:
    unique_nodes.add(n1)
    unique_nodes.add(n2)

len(unique_nodes)

15374

In [42]:
%%time
all_potential_recommendations = find_nodes_at_distance_2(graph_del, unique_nodes)

CPU times: user 1.34 s, sys: 906 ms, total: 2.25 s
Wall time: 7.33 s


In [43]:
len(all_potential_recommendations)

1019091

In [44]:
for rec in list(all_potential_recommendations)[:len(ground_truth)]:
    
    # add to ground truth also the potential nodes
    n1 = rec[0]
    n2 = rec[1]
    
    ground_truth.add((n1, n2, 0))

In [45]:
print(len(ground_truth))

25680


### ALS

In [46]:
# first we get the adjacency matrix of tweets
M = graph_del.get_adjacency_sparse()

In [47]:
# here we run the model ALS
model = implicit.als.AlternatingLeastSquares(factors = 10, calculate_training_loss = True,  iterations = 5)

# train the model on a sparse matrix of item/user/confidence weights
model.fit(M)

In [48]:
def predict_ALS(testset, model):
    """
    predict for a list of observations the score for adding/removing a link
    """

    # initialize the empty list
    all_predictions = []

    # scroll the obs
    for n1, n2, w in testset:
        
        # take here the low-dimensional vectors returned by the matrix factorization
        
        array_n1 = model.user_factors[n1,:]
        array_n2 = model.item_factors[n2,:]

        # multiplying these vectors we generate an approximation for the edge score
        one_p = np.dot(array_n1, array_n2)

        all_predictions.append(one_p)
        
    return all_predictions

In [49]:
# generate the predictions
ALS_results = pd.DataFrame(list(ground_truth), columns = ["n1", "n2", "true"])
all_predictions = predict_ALS(ALS_results.values, model)

# add predictions to df
ALS_results["pred"] = all_predictions

# convert predictions to binary values: 0 don't add the edge, 1 add it.
ALS_results["pred"] = ALS_results["pred"].apply(lambda x: round(x))

### ADAMIC ADAR

In [50]:
def compute_ADA(u, v, graph):
    """
    compute adamic-adar from scratch
    """
    
    # set of neighbors of u
    outlinks_from_u = graph.neighbors(u)

    # set of neighbors of v
    inlinks_to_v = graph.neighbors(v)

    # set Z of neighbors of both
    bridges = set(outlinks_from_u).intersection(inlinks_to_v)

    # degree of nodes in set Z
    deg_ = [graph.degree(n) for n in bridges]
    
    # computing the reciprocal in log-scale
    out = [1./np.log2(dd+1) for dd in deg_]

    return sum(out)

### JACCARD

In [51]:
def compute_Jaccard(u,v, graph):
    """
    compute jaccard similarity
    """

    # set of neighbors of u
    outlinks_from_u = graph.neighbors(u)

    # set of neighbors of v
    inlinks_to_v = graph.neighbors(v)

    # intesection of the two sets
    num = set(outlinks_from_u).intersection(inlinks_to_v)
    
    # union of the two sets
    den = set(outlinks_from_u).union(inlinks_to_v)
    
    # final division
    if len(den) == 0:
        out = 0
    else:
        out = len(num)/len(den)
    
    return out

In [52]:
%%time 
# Dataframe to store the preductions for each edge
ADA_results = pd.DataFrame()
Jaccard_results = pd.DataFrame()

# Iterate over the pairs of nodes of the ground_truth set
for n1, n2, val in ground_truth:
    ADA_results = ADA_results.append({'source': n1, 'destination': n2, 'true': val, 
                                      'pred': compute_ADA(n1, n2, graph_del)}, ignore_index = True)
    Jaccard_results = Jaccard_results.append({'source': n1, 'destination': n2, 'true': val, 
                                              'pred': compute_Jaccard(n1, n2, graph_del)}, ignore_index = True)
    
# Round the predictions
ADA_results['pred'] = ADA_results['pred'].apply(lambda x: round(x))
Jaccard_results['pred'] = Jaccard_results['pred'].apply(lambda x: round(x))

CPU times: user 1min 25s, sys: 3.8 s, total: 1min 29s
Wall time: 1min 46s


In [53]:
ADA_results.head()

,destination,pred,source,true
0,13828.0,0,137.0,1.0
1,23069.0,0,607.0,1.0
2,19233.0,0,7016.0,0.0
3,21201.0,0,16440.0,0.0
4,4863.0,0,3523.0,1.0


In [54]:
Jaccard_results.head()

,destination,pred,source,true
0,13828.0,0,137.0,1.0
1,23069.0,0,607.0,1.0
2,19233.0,0,7016.0,0.0
3,21201.0,1,16440.0,0.0
4,4863.0,0,3523.0,1.0


### PAGERANK


In [55]:
PR_dict = {}

for n1, _, _ in ground_truth:
    if n1 not in PR_dict.keys():
        pr = enumerate(graph_del.personalized_pagerank(reset_vertices = node))
        PR_dict[n1] = sorted(pr, key = lambda x: x[1], reverse = True)[:10]

In [56]:
PageRank_results = pd.DataFrame()

# PageRank algorithm
for n1, n2, val in ground_truth:
    predictions = [pagerank for (pagerank, node2) in PR_dict[n1] if node2 == n2]
    if len(predictions) != 0:
        PageRank_results = PageRank_results.append({'source': node, 'destination': node2, 'true': val, 
                                                    'pred': predictions}, ignore_index = True)
    else:
        PageRank_results = PageRank_results.append({'source': node, 'destination': node2, 'true': val, 
                                                    'pred': 0}, ignore_index = True)
    
PageRank_results['pred'] = PageRank_results['pred'].apply(lambda x: round(x))

### EVALUATION

Accuracy 

In [57]:
# ALS
correct = ALS_results[ALS_results['pred'] == ALS_results['true']]
ALS_accuracy = len(correct)/len(ALS_results)

# ADA 
correct = ADA_results[ADA_results['pred'] == ADA_results['true']]
ADA_accuracy = len(correct)/len(ADA_results)

# Jaccard
correct = Jaccard_results[Jaccard_results['pred'] == Jaccard_results['true']]
Jaccard_accuracy = len(correct)/len(Jaccard_results)

# PageRank
correct = PageRank_results[PageRank_results['pred'] == PageRank_results['true']]
PageRank_accuracy = len(correct)/len(PageRank_results)

print('The accuracy for algorithms is:\n')
print('Adamic-Adar %.4f' %(ADA_accuracy))
print('Jaccard %.4f' %(Jaccard_accuracy))
print('PageRank %.4f' %(PageRank_accuracy))
print('Alternative Least Squares %.4f' %(ALS_accuracy))

The accuracy for algorithms is:

Adamic-Adar 0.4968
Jaccard 0.4173
PageRank 0.5000
Alternative Least Squares 0.5000


Average Precision

In [58]:
from sklearn.metrics import average_precision_score
# ALS
ALS_avg_precision = average_precision_score(ALS_results['true'], ALS_results['pred'])

# ADA 
ADA_avg_precision = average_precision_score(ADA_results['true'], ADA_results['pred'])

# Jaccard
Jaccard_avg_precision = average_precision_score(Jaccard_results['true'], Jaccard_results['pred'])

# PageRank
PageRank_avg_precision = average_precision_score(PageRank_results['true'], PageRank_results['pred'])

print('The average precision for algorithms is:\n')
print('Adamic-Adar %.4f' %(ADA_avg_precision))
print('Jaccard %.4f' %(Jaccard_avg_precision))
print('PageRank %.4f' %(PageRank_avg_precision))
print('Alternative Least Squares %.4f' %(ALS_avg_precision))

The average precision for algorithms is:

Adamic-Adar 0.4999
Jaccard 0.4985
PageRank 0.5000
Alternative Least Squares 0.5000


## 5. PREDICT LINKS USING A CLASSIFIER

### CREATE THE RETWEET GRAPH

In [109]:
# Retweet graph 
df_retweets = df_tweets_100[~df_tweets_100['retweeted_status'].isnull()]

In [110]:
df_graph = pd.DataFrame(columns = ["source", "destination"])

# add source nodes
df_graph["source"] = df_retweets['user'].apply(lambda x: x['screen_name'])

# add destination nodes
df_graph["destination"] =  df_retweets['retweeted_status'].apply(lambda x: x['user']['screen_name'])

# add interaction metrics
df_graph["retweets"] = df_retweets['retweeted_status'].apply(lambda x: x['retweet_count'])
df_graph["favourites"] = df_retweets['retweeted_status'].apply(lambda x: x['favorite_count'])
df_graph["quotes"] = df_retweets['retweeted_status'].apply(lambda x: x['quote_count'])
df_graph["replies"] = df_retweets['retweeted_status'].apply(lambda x: x['reply_count'])

# edge presence
df_graph["edge"] = [1]*len(df_graph)

In [111]:
df_graph.drop_duplicates(subset = ["source", "destination"], inplace = True)

In [112]:
df_graph.head()

,source,destination,retweets,favourites,quotes,replies,edge
0,888Paula,SDPNorthEast,1,1,0,0,1
1,tchevalier10,SenKamalaHarris,2690,35937,221,502,1
2,DPaton4,caljspencer,1090,10770,510,86,1
6,jkcheney,thatalicewu,3836,11260,1328,172,1
7,_annalisekc,leensssss,231,1372,13,3,1


In [113]:
tuples = [tuple(x) for x in df_graph[["source", "destination"]].values]
graph = Graph.TupleList(tuples, directed = False)

In [114]:
print(len(graph.es))

64203


In [115]:
def find_nodes_at_distance_2(graph):
    """
    starting from a graph this function returns all the nodes at distance 2
    """
    
    all_potential_recommendations = set()
    
    for n1 in graph.vs:
        
        # all the nodes at distance 1
        nodes_at_most_distant_1 = set(graph.neighborhood(vertices = n1, order = 1))#, mode = 'out'))

        # all the nodes at distance 1 and distance 2
        nodes_at_most_distant_2 = set(graph.neighborhood(vertices = n1, order = 2))#, mode = 'out'))
        
        # only the nodes at distance 2
        only_nodes_at_distance_2 = nodes_at_most_distant_2 - nodes_at_most_distant_1
        
        # check if empty set
        if len(only_nodes_at_distance_2) > 0:

            for n2 in only_nodes_at_distance_2:
                
                all_potential_recommendations.add((graph.vs[n1.index]['name'], graph.vs[n2]['name']))
            
    return all_potential_recommendations

In [116]:
%%time
all_potential_recommendations = find_nodes_at_distance_2(graph)

CPU times: user 1min 10s, sys: 7.31 s, total: 1min 17s
Wall time: 1min 23s


In [117]:
all_potential_recommendations = list(all_potential_recommendations)[:len(df_graph)]
print(len(all_potential_recommendations))

64203


In [118]:
df_info = pd.DataFrame()

df_info['users'] = df_retweets['retweeted_status'].apply(lambda x: x['user']['screen_name'])
df_info['retweets'] = df_retweets['retweeted_status'].apply(lambda x: x['retweet_count'])
df_info['favourites'] = df_retweets['retweeted_status'].apply(lambda x: x['favorite_count'])
df_info['quotes'] = df_retweets['retweeted_status'].apply(lambda x: x['quote_count'])
df_info['replies'] = df_retweets['retweeted_status'].apply(lambda x: x['reply_count'])

df_info.drop_duplicates("users", inplace = True)

In [119]:
n1_nodes = []
n2_nodes = []

rets = []
favs = []
quotes = [] 
replies = []

for rec in all_potential_recommendations:     
    n1 = rec[0]
    n2 = rec[1]
    
    n1_nodes.append(n1)
    n2_nodes.append(n2)
    
    if n1 in df_info['users'].values:
        rets.append(df_info[df_info['users'] == n1]['retweets'])
        favs.append(df_info[df_info['users'] == n1]['favourites'])
        quotes.append(df_info[df_info['users'] == n1]['quotes'])
        replies.append(df_info[df_info['users'] == n1]['replies'])
        
    elif n2 in df_info['users'].values:
        rets.append(df_info[df_info['users'] == n2]['retweets'])
        favs.append(df_info[df_info['users'] == n2]['favourites'])
        quotes.append(df_info[df_info['users'] == n2]['quotes'])
        replies.append(df_info[df_info['users'] == n2]['replies'])
        
    else:
        rets.append(0)
        favs.append(0)
        quotes.append(0)
        replies.append(0)

In [101]:
df_potential = pd.DataFrame()
df_potential["source"] = n1_nodes
df_potential["destination"] = n2_nodes
df_potential["retweets"] = rets
df_potential["favourites"] = favs
df_potential["quotes"] = quotes
df_potential["replies"] = replies
df_potential["edge"] = [0]*len(df_potential)

In [120]:
df_graph = df_graph.append(df_potential, ignore_index = True)

# Remove recommendations to oneself
df_graph = df_graph[df_graph['source'] != df_graph['destination']]
df_graph

,source,destination,retweets,favourites,quotes,replies,edge
0,888Paula,SDPNorthEast,1,1,0,0,1
1,tchevalier10,SenKamalaHarris,2690,35937,221,502,1
2,DPaton4,caljspencer,1090,10770,510,86,1
3,jkcheney,thatalicewu,3836,11260,1328,172,1
4,_annalisekc,leensssss,231,1372,13,3,1
...,...,...,...,...,...,...,...
128401,reginaa_xoo,terrikibiriti,0,0,0,0,0
128402,pefnic,Barbielynn01,0,0,0,0,0
128403,pacheco_wilson,Gshu1tz,0,0,0,0,0
128404,Time2change17,Heather03807097,0,0,0,0,0


In [121]:
## Shuffle dataframe
from sklearn.utils import shuffle
df_graph = shuffle(df_graph)

In [122]:
df_graph['edge'].value_counts()

0    64203
1    63774
Name: edge, dtype: int64

### Train - Test Split

In [123]:
## Arrays

X = df_graph[["retweets", "favourites", "quotes", "replies"]].values
y = df_graph["edge"].values

print(X.shape)
print(y.shape)

(127977, 4)
(127977,)


In [124]:
from sklearn.model_selection import train_test_split
# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [125]:
from sklearn import svm
## Support Vector Machine Classifier
SVM = svm.LinearSVC()

# Train
SVM.fit(X_train, y_train)

# Test
y_pred = SVM.predict(X_test)

/home/pdp/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [126]:
from sklearn.metrics import accuracy_score

print('SVM classifier accuracy: ' + str(accuracy_score(y_test, y_pred)))
print('SVM average precision score: ' + str(average_precision_score(y_test, y_pred)))

SVM classifier accuracy: 0.9628848257540241
SVM average precision score: 0.9597409173041738
